In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from riotwatcher import LolWatcher, ApiError


In [4]:
lol_watcher = LolWatcher('RGAPI-a2e5141b-98d9-427a-905c-594767fb16cf')
my_region = 'euw1'


In [18]:
list1 = lol_watcher.spectator.featured_games(my_region)
list1 = pd.DataFrame(list1)

In [26]:
list2 = lol_watcher.league.entries(my_region,
                                   queue='RANKED_SOLO_5x5',
                                   tier='GOLD',
                                   division='I',
                                   page=1
)
list2 = pd.DataFrame(list2)

In [33]:
list2.head()

,leagueId,queueType,tier,rank,puuid,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,7016b98b-2cfb-466a-a925-9889f3ca4cef,RANKED_SOLO_5x5,GOLD,I,diKw5K4Wx12xRvNThME5uAXuUFRW80crX2K98xzYf6RZ1h...,33,16,13,False,False,False,False
1,592c73a0-967f-4954-8a55-2db4e68737da,RANKED_SOLO_5x5,GOLD,I,cLun3lJMFwCQrTen8ktxxVuM1TjkVAVEKaZpjz_uVPuPDt...,43,57,75,False,False,False,False
2,33128fdc-17d0-4848-9254-77a6d09d599f,RANKED_SOLO_5x5,GOLD,I,PTiI7z4WR1AeioPPA5uF7iBGmc9UZY_vW0m4K7iCbzA_QZ...,57,91,116,False,False,False,False
3,973447cf-e254-4966-89ce-1d1d2635439e,RANKED_SOLO_5x5,GOLD,I,fu_6SGZcFK2PzInlO-UFSKMEgpFBKYoB3QaxNB5O5v8D3y...,45,170,177,False,False,False,False
4,164d8791-9ed9-44b9-9260-48a67083d1f9,RANKED_SOLO_5x5,GOLD,I,-Wjmh275neDtcanXt4n_blxhTxQcEi-Wfc8_LRIGGl13Lv...,39,74,87,False,False,False,False
